In [2]:
def load_queries(filename):
    queries = {}
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            query_id, query_text = line.strip().split('\t')
            queries[query_id] = query_text
    return queries

queries = load_queries('../data/test_queries/msmarco-test2020-queries.tsv')

In [75]:
def load_qrels(filename):
    qrels = {}
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            query_id, _, doc_id, relevance = line.strip().split()
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    return qrels

qrels = load_qrels('../data/test_queries/2020qrels-pass.txt')


In [1]:
import time

from input_output.index_io import load_document_index, load_inverted_index_binary, load_lexicon

# Load inverted index and lexicon
inverted_index_start = time.time()
print("Loading the inverted index")
inverted_index = load_inverted_index_binary('../data/inverted_index_8841823.bin')
print(f"Inverted index loaded in {time.time() - inverted_index_start} s")
total_docs = len(inverted_index)  # Adjust this if needed

lexicon_start = time.time()
print("Loading lexicon")
lexicon = load_lexicon('../data/lexicon.txt')
print(f"Lexicon loaded in {time.time() - lexicon_start} s")

document_index_start = time.time()
print("Loading document index")
document_index = load_document_index("../data/document_index.txt")
print(f"Document index loaded in {time.time() - document_index_start} s")

Loading the inverted index
Inverted index loaded in 111.32967877388 s
Loading lexicon
Lexicon loaded in 0.03603100776672363 s
Loading document index
Document index loaded in 5.946610689163208 s


In [73]:
import math
from statistics import mean
from query_processing.process_query import process_query

ndcg = []
nb_queries = 0
elapsed_time = 0
for qid, query in queries.items():
    if qid not in qrels:
        continue
    k = len(qrels[qid])
    querying_start = time.time()
    results = process_query(query, inverted_index, lexicon, document_index, total_docs, ranking='tfidf')[:20]
    elapsed_time += time.time() - querying_start
    nb_queries += 1
    results = {docid for docid,_ in results}
    weighted_assessed_run = []
    for docid in results:
        if docid in qrels[qid]:
            weighted_assessed_run.append(qrels[qid][docid])
        else:
            weighted_assessed_run.append(0)
    if len(weighted_assessed_run) == 0 :
        ndcg.append(0)
        continue

    dcg = weighted_assessed_run[0]
    count = 1
    for weight in weighted_assessed_run[1:]:
        dcg += weight / math.log2(count +1) 
        count += 1

    expected_results = list(qrels[qid].values())
    weighted_assessed_ideal_run = sorted(expected_results)[20:]
    idcg = weighted_assessed_ideal_run[0]
    count = 1
    for weight in weighted_assessed_ideal_run[1:]:
        idcg += weight / math.log2(count + 1) 
        count += 1

    ndcg.append(dcg/idcg)

print(mean(ndcg))
print(elapsed_time)
print(elapsed_time/nb_queries)


0.12118458706471585
62.71790075302124
1.1614426065374304


In [74]:
ndcg = []
nb_queries = 0
elapsed_time = 0
for qid, query in queries.items():
    if qid not in qrels:
        continue
    k = len(qrels[qid])
    querying_start = time.time()
    results = process_query(query, inverted_index, lexicon, document_index, total_docs, ranking='bm25')[:20]
    elapsed_time += time.time() - querying_start
    nb_queries += 1
    results = {docid for docid,_ in results}
    weighted_assessed_run = []
    for docid in results:
        if docid in qrels[qid]:
            weighted_assessed_run.append(qrels[qid][docid])
        else:
            weighted_assessed_run.append(0)
    if len(weighted_assessed_run) == 0 :
        ndcg.append(0)
        continue

    dcg = weighted_assessed_run[0]
    count = 1
    for weight in weighted_assessed_run[1:]:
        dcg += weight / math.log2(count +1) 
        count += 1

    expected_results = list(qrels[qid].values())
    weighted_assessed_ideal_run = sorted(expected_results)[20:]
    idcg = weighted_assessed_ideal_run[0]
    count = 1
    for weight in weighted_assessed_ideal_run[1:]:
        idcg += weight / math.log2(count + 1) 
        count += 1

    ndcg.append(dcg/idcg)

print(mean(ndcg))
print(elapsed_time)
print(elapsed_time/nb_queries)


0.1872832393372296
109.60219550132751
2.0296702870616206
